In [0]:
%sql
CREATE OR REPLACE VIEW d_dnst_030_gold.pbi_views.v_sellout_customers_with_duplicates AS
  /* Metadata
  {
    "operation_type": "virtualize",
    "dependencies": ["`d_dnst_020_silver.silver_03.032_sellout_customers_ext_matched`"]
  }
  */
  WITH ranked AS (
    SELECT
        customer_number               AS `Id at Wholesaler`,
        customer_address              AS `Customer Address`,
        customer_city                 AS `Customer City`,
        customer_postal_code          AS `Customer Postal Code`,
        customer_country              AS `Customer Country`,
        customer_name                 AS `Customer Name`,
        wholesaler_name               AS `Wholesaler Name`,
        vat_number                    AS `Vat Number`,
        LPAD(direct_c4S_account_id, 10, '0') AS `C4s Account Id`,
        wholesaler_id                 AS `Wholesaler Id`,
        Place_ID                      AS `Place Id`,
        customer_number_Occurrence    AS `Occurrence`,
        concat(customer_number,wholesaler_id ) AS `Id at Wholesaler + Wholesaler Id`,
        dim_sellout_cust_id           AS dim_sellout_cust_id,
 
        -- 1. Assign a rank (RN) within each duplicate group.
        ROW_NUMBER() OVER (
          PARTITION BY
            dim_sellout_cust_id,
            LPAD(direct_c4S_account_id, 10, '0') -- The "C4s Account Id" value
          ORDER BY
            customer_number DESC -- Prioritize the record with the highest occurrence count.
        ) AS RN
    FROM d_dnst_020_silver.silver_03.`032_sellout_customers_ext_matched`
  )

    -- 2. Select all columns for the top-ranked record for each group (RN = 1).
  SELECT
    `Id at Wholesaler`,
    `Customer Address`,
    `Customer City`,
    `Customer Postal Code`,
    `Customer Country`,
    `Customer Name`,
    `Wholesaler Name`,
    `Vat Number`,
    `C4s Account Id`,
    `Wholesaler Id`,
    `Place Id`,
    `Occurrence`,
    `Id at Wholesaler + Wholesaler Id`,
    dim_sellout_cust_id
  FROM ranked
  WHERE RN = 1